<a href="https://colab.research.google.com/github/CUH-MCA/signlanguageML/blob/main/signlanguageproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Sign Language Recognition Project </h1>

<h3>Importing Libraries

In [1]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt

### Rest of The Code ☁ -->

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
data = tf.keras.utils.image_dataset_from_directory('asl_dataset')

NotFoundError: ignored

In [4]:
data_iterator = data.as_numpy_iterator()

NameError: ignored

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [ ]:
train_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(16,(3,3),activation = "relu" , input_shape = (180,180,3)) ,
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(32,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64,(3,3),activation = "relu") ,  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128,(3,3),activation = "relu"),  
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(), 
#     tf.keras.layers.Dense(550,activation="relu"),      #Adding the Hidden layer
#     tf.keras.layers.Dropout(0.1,seed = 2019),
#     tf.keras.layers.Dense(400,activation ="relu"),
#     tf.keras.layers.Dropout(0.3,seed = 2019),
#     tf.keras.layers.Dense(300,activation="relu"),
#     tf.keras.layers.Dropout(0.4,seed = 2019),
#     tf.keras.layers.Dense(200,activation ="relu"),
#     tf.keras.layers.Dropout(0.2,seed = 2019),
#     tf.keras.layers.Dense(5,activation = "softmax")  #Adding the Output Layer
# ])

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=10, validation_data=val, callbacks=[tensorboard_callback])

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
ig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

In [ ]:
import cv2

In [ ]:
img = cv2.imread('WIN_20230430_21_08_04_Pro.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
from tensorflow.keras.models import load_model
# model.save(os.path.join('models','imageclassifier.h5'))